# CS295/395: Secure Distributed Computation
## In-Class Exercise, week of 10/31/2022

In [1]:
import hashlib

In [9]:
for i in range(10):
    print(hashlib.sha256(int.to_bytes(i, 3, 'little')).hexdigest())

709e80c88487a2411e1ee4dfb9f22a861492d20c4765150c0c794abd70f8147c
fb50dc0717ff266cf9baf82b1ce7a1c2ef6d9247859680b11a19fb7077f5f222
070b13c004e9475fa155cd84d90de9f87db1b7dea6707329638a6ce66043d246
3e4479e6b5a0a3bed1440325ca4bb09327a1dfb39c28bdd59de53a687ade0105
3ee0d7c44a58950b18c1e01b912ed0e08549ff905fbf7010318a44d3e2efb8db
69c492e24afa52c556de5ea70e1b5d7da3f0bc7a5a7fd1d413444c1a42a7e9b3
412c45d603453521684df8f679c7f68f0f22dd7697d37170385c8c7931636298
63dc0f142f3c734e10f668cb28083535f47159e64c350a0b646416e847a2da2f
736722f22cbebd49059bbc5d24c1bf71acabcabc8a4ba8d3776392d54728dc91
bdb8cd2e44905845887b8f16d3a0c9f33103496315703d1838f94d0c4eccb1b2


## Question 1

Implement a *commitment scheme* using `hashlib.sha256`.

In [15]:
class CommitmentSender:
    def __init__(self, receiver):
        self.receiver = receiver
    
    def commit(self, string):
        
        self.string = string
        self.hash = hashlib.sha256(bytes(self.string)).hexdigest()
        self.receiver.receive_commit(self.hash)
        
    def open(self):
        self.receiver.receive_open(self.string)

class CommitmentReceiver:
    def receive_commit(self, received_hash):
        # save the hash I received so I can check it later on
        self.committed_hash = received_hash
        
    def receive_open(self, string):
        # recompute the hash of the string I received and verify that it matches the hash I received earlier
        assert(self.committed_hash == hashlib.sha256(bytes(string)).hexdigest())

r = CommitmentReceiver()
s = CommitmentSender(r)
s.commit(50)
s.open()

## Question 2

Implement a *cheating* sender for the commitment scheme from question 1. The receiever should be able to tell that the cheating sender has cheated.

In [18]:
class CheatingCommitmentSender(CommitmentSender):

    def open(self, new_string):
        self.receiver.receive_open(new_string)

r = CommitmentReceiver()
s = CheatingCommitmentSender(r)
s.commit(5)
s.open(6)

AssertionError: 

## Interactive Zero-Knowledge Protocol for Graph Coloring

**Statement**: For (public) graph $G$, Prover knows a valid 3-coloring.

**Witness**: The valid 3-coloring.

One iteration of the protocol:
- **Commit**: Prover shuffles the coloring, to obtain a new valid 3-coloring. Prover commits to the color of each vertex in the graph under the shuffled coloring (e.g. by hashing the result of concatenating the color with a random number). Prover sends the commitment to Verifier.
- **Challenge**: The Verifier picks a random edge in the graph and constructs a challenge consisting of the two vertices connected by the edge. Verifier sends the challenge to Prover.
- **Response**: The Prover opens the commitment for the two vertices in the challenge (e.g. by revealing their colors and the random number used in committing) to the Verifier.
- **Check**: The Verifier checks that (1) the commitment was valid for the vertices in the response; (2) the two colors are different.

The probability of a cheating Prover "getting away with it" in one iteration is $(E-1)/E$ where $E$ is the number of edges.
Prover and Verifier run this protocol $n$ times. The total probability that the cheating prover "gets away with it" is $((E-1)/E)^n$.

## Question 3

Design a similar protocol for proving statements *encoded as boolean circuits* using the MPC-in-the-head approach.

Statement: For a circuit C, the prover knows values w for the inputs such that C outputs True (1)

Witness: The value of the inputs to the circuit (w)

One iteration of the protocol:
- Setup:
  - Prover splits witness into secret shares
  - Prover runs the GMW protocol using P imaginary parties with secret shares as input, and saves the views of all of them
- Commit:
  - Prover commits to each parties view individually and sends commitments to the verifier
- Challenge: Verifier picks p1, p2 from P at random
- Response: Prover opens the commitments for the views p1 and p2
- Check: Verifier checks that:
  - The commitment was valid for p1, p2
  - The output of the circuit was True (1)
  - The views of p1 and p2 were consistent with each other
    - Each message recieved by p_i hsould be a message that would have been generated by the protocol at the appropriate round and sent to p_i

- The probability that the prover cheats and gets away with it is $(p-1)p$ where p is the number of parties. We run N iterations of the protocol to reduce this probability to $\frac{p-1}{p}^n$

## Rewinding
The simulator can stop and go back in time during the simulation. This lets it cheat.

- Commit: Prover commits to a random solution (might be invalid)
- Challenge: Verifier randomly issues a challenge
  - If this challenge would have resulted in the verifier catching P cheating, then the verifier rewinds to commit and starts again
- Respond: as in the protocol
- Check: as in the protocol

## Question 4

Modify your protocol from Question 1 into a *non-interactive* (NIZK) protocol.

- Prover does the commit, prover does the challenge, prover construct response, verifier checks at the end.
- Now we can do all the "proving" at once and package it up and distribute it.

We will use the Fiat-Shamir heuristic to transform the protocol.
- We can get some shared randomness by hashing a value the prover doesn't control.
- Shared randomness - a random value that the verifier can check that it was actually random and not chosen maliciously by the prover.
- Could be done by hashing the commitment: H(C|round_num)